In [24]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from IPython.display import FileLink, display
import qrcode, os
from datetime import datetime

def render_po_si_form_prefilled(cfg=None):
    cfg = cfg or {}
    title = cfg.get("title","PURCHASE ORDERS & SALES INVOICES FORM")
    filename = cfg.get("filename","po_si_form_prefilled_a4.pdf")
    page_size = cfg.get("page_size", A4)
    margins_mm = cfg.get("margins_mm", 10)
    pad_mm = cfg.get("pad_mm", 3)
    label_font = cfg.get("label_font", ("Helvetica","Helvetica-Bold"))
    label_font_sizes = cfg.get("label_font_sizes",(10,7.2))
    value_font_max = cfg.get("value_font_max", 11.5)
    title_font_sizes = cfg.get("title_font_sizes",(17,10))
    base_row_height_mm = cfg.get("row_height_mm", 4.6)
    gap_mm = cfg.get("gap_mm", 1.6)
    qr_mm = cfg.get("qr_mm", 14)
    qr_border = cfg.get("qr_border", 1)
    tables_extra_top_gap_mm = cfg.get("tables_extra_top_gap_mm", 6)

    w,h = page_size
    m = margins_mm*mm
    pad = pad_mm*mm
    left,right = m, w-m
    usable_w = right-left
    gap = gap_mm*mm
    extra_gap = tables_extra_top_gap_mm*mm

    HEIGHTS = {"single":10*mm,"multi":14*mm}
    FIELDS = [
        ([({"key":"form_name","label":"Form Name"},0.30),
          ({"key":"current_form_qr_code","label":"Form QR Code"},0.25),
          ({"key":"generated_on","label":"Generated On"},0.22),
          ({"key":"generated_by","label":"Generated By"},0.23)],"single"),
        ([({"key":"location","label":"Location"},0.50),
          ({"key":"previous_recorder","label":"Previous Recorder"},0.50)],"single"),
        ([({"key":"date_iso","label":"Date (YYYY-MM-DD)"},0.50),
          ({"key":"date_name","label":"Date (Name e.g., Jan. 01, 2025)"},0.50)],"single"),
    ]

    def fit_font_size(text,font_name,max_font,min_font,box_width):
        size=max_font
        while size>min_font and pdfmetrics.stringWidth(text,font_name,size)>box_width:
            size-=0.5
        return max(min_font,size)

    def draw_scaled_title(c,text,top_y,side_margin=m,max_font=title_font_sizes[0],min_font=title_font_sizes[1]):
        box_w=w-2*side_margin
        fs=fit_font_size(text,label_font[1],max_font,min_font,box_w)
        c.setFont(label_font[1],fs)
        c.setFillColor(colors.black)
        c.drawCentredString(w/2,top_y,text)
        return fs

    def resolve_height(height_spec,cols):
        if isinstance(height_spec,(int,float)): return float(height_spec)
        if isinstance(height_spec,str) and height_spec in HEIGHTS: return HEIGHTS[height_spec]
        return HEIGHTS["multi"] if len(cols)==1 else HEIGHTS["single"]

    def draw_row(c,top_y,height,cols,prefill):
        c.setLineWidth(1.1)
        c.setStrokeColor(colors.black)
        c.rect(left,top_y-height,usable_w,height,stroke=1,fill=0)
        x=left; acc=0.0
        for i,(field,ratio) in enumerate(cols):
            acc+=ratio
            if i<len(cols)-1:
                x_next=left+usable_w*acc
                c.line(x_next,top_y-height,x_next,top_y)
            col_w=usable_w*ratio
            label=field.get("label") or ""
            key=field.get("key") or ""
            fs=fit_font_size(label,label_font[0],label_font_sizes[0],label_font_sizes[1],col_w-2*pad)
            c.setFont(label_font[0],fs); c.setFillColor(colors.black)
            c.drawString(x+pad,top_y-pad-fs*0.2,label)
            val=str(prefill.get(key,""))
            if val:
                vfs=fit_font_size(val,label_font[0],value_font_max,8,col_w-2*pad)
                c.setFont(label_font[1],vfs)
                c.drawString(x+pad,top_y-6.8*mm,val)
            x=left+usable_w*acc

    def _make_qr_image(path, value, box_mm=qr_mm, border=qr_border):
        qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=4, border=border)
        qr.add_data(value); qr.make(fit=True)
        img = qr.make_image(fill_color="black", back_color="white")
        img.save(path)
        return box_mm*mm

    def draw_section_caption(c, text, x, y):
        c.setFont(label_font[1], 9.0)
        c.setFillColor(colors.black)
        c.drawString(x, y, text)

    def draw_table(c, x, y_top, col_widths_mm, headers, n_rows, row_h_mm, stroke=0.4):
        col_ws = [wmm*mm for wmm in col_widths_mm]
        total_w = sum(col_ws)
        c.setLineWidth(stroke)
        c.setStrokeColor(colors.black)
        c.rect(x, y_top - (row_h_mm*(n_rows+1)), total_w, row_h_mm*(n_rows+1), stroke=1, fill=0)
        cx = x
        for cw in col_ws[:-1]:
            cx += cw
            c.line(cx, y_top - (row_h_mm*(n_rows+1)), cx, y_top)
        for r in range(n_rows+1):
            yy = y_top - (row_h_mm*r)
            c.line(x, yy, x+total_w, yy)
        c.setFont(label_font[1], 7.6)
        cur_x = x
        for i,hdr in enumerate(headers):
            c.drawString(cur_x+1.4*mm, y_top - row_h_mm + 1.8*mm, hdr)
            cur_x += col_ws[i]
        c.setFont("Helvetica", 7.4)

    def ensure_date_fields(prefill):
        iso = str(prefill.get("date_iso","")).strip()
        name = str(prefill.get("date_name","")).strip()
        if iso and not name:
            try:
                dt = datetime.strptime(iso, "%Y-%m-%d")
                prefill["date_name"] = dt.strftime("%b. %d, %Y")
            except Exception:
                pass
        return prefill

    c=canvas.Canvas(filename,pagesize=page_size)
    c.setTitle(title)
    prefill = ensure_date_fields(cfg.get("prefill",{}))

    title_y=h-m-5*mm
    draw_scaled_title(c,title,title_y)

    cursor = title_y - 3*mm

    qr_value = str(prefill.get("current_form_qr_code","")).strip()
    if qr_value:
        qr_path = "po_si_qr_prefilled.png"
        qr_box = _make_qr_image(qr_path, qr_mm)
        x_qr = (w - qr_box)/2
        y_qr = cursor - qr_box - 1.6*mm
        c.drawImage(qr_path, x_qr, y_qr, width=qr_box, height=qr_box, preserveAspectRatio=True, mask='auto')
        c.setFont(label_font[0],7.2)
        c.drawCentredString(w/2, y_qr - 1.6*mm, qr_value)
        cursor = y_qr - 4.5*mm

    for cols,height_spec in FIELDS:
        h_row=resolve_height(height_spec,cols)
        draw_row(c,cursor,h_row,cols,prefill)
        cursor-=h_row

    cursor -= extra_gap

    po_headers = ["PO No","Plate No","Location","Driver","Product","Quantity","Price","Amount"]
    po_col_mm  = [  20,     18,        31,        34,      19,       12,        12,     40]

    inv_headers = ["Invoice ID","Customer Name","TIN","Product","Qty (L)","Price","Amount"]
    inv_col_mm  = [32,          38,             24,   22,       14,       16,     40]  # sums to 186; Amount widest


    static_offsets = 6*mm + 6*mm + HEIGHTS["single"] + 3*mm
    available_for_tables = (cursor - m) - static_offsets
    total_table_rows = (1+15) + (1+15)
    row_h = base_row_height_mm*mm
    max_row_h_that_fits = available_for_tables / total_table_rows
    row_h = min(row_h, max(3.6*mm, max_row_h_that_fits))

    draw_section_caption(c, "PURCHASE ORDERS (15 rows)", left, cursor-1.6*mm)
    cursor -= 6*mm
    draw_table(c, left, cursor, po_col_mm, po_headers, 15, row_h)
    cursor -= ((1+15)*row_h + 3.5*mm)

    draw_section_caption(c, "SALES INVOICES (15 rows)", left, cursor-1.6*mm)
    cursor -= 6*mm
    draw_table(c, left, cursor, inv_col_mm, inv_headers, 15, row_h)
    cursor -= ((1+15)*row_h + 3*mm)

    bottom_fields = [
        ({"key":"cashier_employee_number","label":"Cashier Employee Number"}, 0.50),
        ({"key":"recorder_employee_number","label":"Recorder Employee Number"}, 0.50),
    ]
    draw_row(c, cursor, HEIGHTS["multi"], bottom_fields, prefill)

    c.save()
    try: display(FileLink(filename))
    except: pass
    return os.path.abspath(filename)

render_po_si_form_prefilled({
    "tables_extra_top_gap_mm": 6,
    "prefill":{
        "form_name":"Purchase Orders & Sales Invoices",
        "current_form_qr_code":"1000012345707",
        "generated_on":"2025-10-05",
        "generated_by":"JEF Quickeln",
        "location":"Main Depot - Tagbilaran",
        "previous_recorder":"Ana D.",
        "date_iso":"2025-10-05",
        "date_name":"Oct. 05, 2025",
        "cashier_employee_number":"",
        "recorder_employee_number":""
    }
})


h:\github4\jefstore-gasstations-backend\reportlab\daily-form\po_si_form_prefilled_a4.pdf

'h:\\github4\\jefstore-gasstations-backend\\reportlab\\daily-form\\po_si_form_prefilled_a4.pdf'